In [0]:
%python
# Fetching the csv file from the blob storage
storage_account_name = "codingchallengedlacc"
container_name = "my-container"
storage_account_key = "hMW9uKwa01lepiUgz0tGQd9P9UKbeVq29mCY5wCz2GIPpsMELvdjGaAcJMmDjsiX8RHCznioSigW+AStcpQIgQ=="

# Unmount the directory if it is already mounted
if any(mount.mountPoint == "/mnt/superstore" for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount("/mnt/superstore")

# Mount dl Storage
dbutils.fs.mount(
    source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point="/mnt/superstore",
    extra_configs={
        f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key
    }
)

True

In [0]:
#verifying the mount
display(dbutils.fs.ls("/mnt/superstore"))

path,name,size,modificationTime
dbfs:/mnt/superstore/Global_Superstore2.csv,Global_Superstore2.csv,12089916,1734606845000


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SuperstoreETL").getOrCreate()

# Load the dataset
file_path = "/mnt/superstore/Global_Superstore2.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show(5)


+------+---------------+----------+----------+------------+-----------+----------------+-----------+-------------+---------------+-------------+-----------+------+-------+----------------+----------+------------+--------------------+--------+--------+--------+--------+-------------+--------------+
|Row ID|       Order ID|Order Date| Ship Date|   Ship Mode|Customer ID|   Customer Name|    Segment|         City|          State|      Country|Postal Code|Market| Region|      Product ID|  Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|Shipping Cost|Order Priority|
+------+---------------+----------+----------+------------+-----------+----------------+-----------+-------------+---------------+-------------+-----------+------+-------+----------------+----------+------------+--------------------+--------+--------+--------+--------+-------------+--------------+
| 32298| CA-2012-124891|2012-07-31|2012-07-31|    Same Day|   RH-19495|     Rick Hansen|   Consumer|New

In [0]:
#Add Total Cost Column
from pyspark.sql.functions import col

df_transformed = df.withColumn("TotalCost", col("Sales") - col("Profit"))
df.show(5)

+------+---------------+----------+----------+------------+-----------+----------------+-----------+-------------+---------------+-------------+-----------+------+-------+----------------+----------+------------+--------------------+--------+--------+--------+--------+-------------+--------------+
|Row ID|       Order ID|Order Date| Ship Date|   Ship Mode|Customer ID|   Customer Name|    Segment|         City|          State|      Country|Postal Code|Market| Region|      Product ID|  Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|Shipping Cost|Order Priority|
+------+---------------+----------+----------+------------+-----------+----------------+-----------+-------------+---------------+-------------+-----------+------+-------+----------------+----------+------------+--------------------+--------+--------+--------+--------+-------------+--------------+
| 32298| CA-2012-124891|2012-07-31|2012-07-31|    Same Day|   RH-19495|     Rick Hansen|   Consumer|New

In [0]:
#Convert Sales and Profit to Float

df_transformed = df_transformed.withColumn("Sales", col("Sales").cast("float"))
df_transformed = df_transformed.withColumn("Profit", col("Profit").cast("float"))

df_transformed.show(5)

+------+---------------+----------+----------+------------+-----------+----------------+-----------+-------------+---------------+-------------+-----------+------+-------+----------------+----------+------------+--------------------+--------+--------+--------+--------+-------------+--------------+------------------+
|Row ID|       Order ID|Order Date| Ship Date|   Ship Mode|Customer ID|   Customer Name|    Segment|         City|          State|      Country|Postal Code|Market| Region|      Product ID|  Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|Shipping Cost|Order Priority|         TotalCost|
+------+---------------+----------+----------+------------+-----------+----------------+-----------+-------------+---------------+-------------+-----------+------+-------+----------------+----------+------------+--------------------+--------+--------+--------+--------+-------------+--------------+------------------+
| 32298| CA-2012-124891|2012-07-31|2012-07-31|

In [0]:
#Filter Rows with Zero or Negative Profit

df_transformed = df_transformed.filter(col("Profit") > 0)
df_transformed.show(5)

+------+--------------+----------+----------+------------+-----------+-------------+---------+-------------+---------------+-------------+-----------+------+-------+----------------+----------+------------+--------------------+--------+--------+--------+--------+-------------+--------------+------------------+
|Row ID|      Order ID|Order Date| Ship Date|   Ship Mode|Customer ID|Customer Name|  Segment|         City|          State|      Country|Postal Code|Market| Region|      Product ID|  Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|Shipping Cost|Order Priority|         TotalCost|
+------+--------------+----------+----------+------------+-----------+-------------+---------+-------------+---------------+-------------+-----------+------+-------+----------------+----------+------------+--------------------+--------+--------+--------+--------+-------------+--------------+------------------+
| 32298|CA-2012-124891|2012-07-31|2012-07-31|    Same Day|   RH-

In [0]:
from pyspark.sql.functions import col, sum
#Group by Category and Region and perform aggregation

# Step 1: Clean the data by filtering out rows with non-numeric values in Sales, Quantity, or Discount
df_clean = df.filter(
    (col("Sales").rlike("^[0-9]*\.?[0-9]+$")) &  # Allow decimal numbers in Sales
    (col("Quantity").rlike("^[0-9]+$")) &  # Allow only integers in Quantity
    (col("Discount").rlike("^[0-9]*\.?[0-9]+$"))  # Allow decimal numbers in Discount
)

# Step 2: Cast the cleaned columns to appropriate numeric types
df_clean = df_clean.withColumn("Sales", col("Sales").cast("float")) \
                   .withColumn("Quantity", col("Quantity").cast("int")) \
                   .withColumn("Discount", col("Discount").cast("float"))

# Step 3: Drop rows with any null values in the numeric columns (after casting)
df_clean = df_clean.dropna(subset=["Sales", "Quantity", "Discount", "Profit"])

# Step 4: Group by Category and Region and perform aggregation
df_transformed = df_clean.groupBy("Category", "Region") \
    .agg(
        sum("Sales").alias("TotalSales"),
        sum("Profit").alias("TotalProfit"),
        sum("Quantity").alias("TotalQuantity"),
        sum("Discount").alias("TotalDiscount")
    )

# Step 5: Show the transformed data
df_transformed.show(5)


+----------+------+------------------+------------------+-------------+-----------------+
|  Category|Region|        TotalSales|       TotalProfit|TotalQuantity|    TotalDiscount|
+----------+------+------------------+------------------+-------------+-----------------+
| Furniture|Canada|10595.279964447021|           2613.24|           78|              0.0|
|Technology|  EMEA|  300854.583026886|17494.443000000036|         2259|189.1000056862831|
| Furniture|  East| 205540.3473367691|         2501.8162|         2151| 90.6000018119812|
|Technology|Africa| 322367.0430994034|  44129.4930000001|         2031|143.1999975964427|
|Technology|  East| 264872.0816922188| 47439.55759999996|         1927|76.30000080168247|
+----------+------+------------------+------------------+-------------+-----------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import col

# Add a Profit Margin Column 
df_transformed = df_transformed.withColumn("ProfitMargin", (col("TotalProfit") / col("TotalSales")) * 100)

# Show the result
df_transformed.show(5)


+----------+------+------------------+------------------+-------------+-----------------+------------------+
|  Category|Region|        TotalSales|       TotalProfit|TotalQuantity|    TotalDiscount|      ProfitMargin|
+----------+------+------------------+------------------+-------------+-----------------+------------------+
| Furniture|Canada|10595.279964447021|           2613.24|           78|              0.0|24.664190174953884|
|Technology|  EMEA|  300854.583026886|17494.443000000036|         2259|189.1000056862831| 5.814916569988444|
| Furniture|  East| 205540.3473367691|         2501.8162|         2151| 90.6000018119812|1.2171898278934408|
|Technology|Africa| 322367.0430994034|  44129.4930000001|         2031|143.1999975964427|13.689207363046899|
|Technology|  East| 264872.0816922188| 47439.55759999996|         1927|76.30000080168247|17.910365372189244|
+----------+------+------------------+------------------+-------------+-----------------+------------------+
only showing top 5 

In [0]:
# Remove Duplicate Rows based on 'Category' and 'Region' 
df_transformed = df_transformed.dropDuplicates(["Category", "Region"])

df_transformed.show(5)


+----------+------+------------------+------------------+-------------+-----------------+------------------+
|  Category|Region|        TotalSales|       TotalProfit|TotalQuantity|    TotalDiscount|      ProfitMargin|
+----------+------+------------------+------------------+-------------+-----------------+------------------+
| Furniture|Canada|10595.279964447021|           2613.24|           78|              0.0|24.664190174953884|
|Technology|  EMEA|  300854.583026886|17494.443000000036|         2259|189.1000056862831| 5.814916569988444|
| Furniture|  East| 205540.3473367691|         2501.8162|         2151| 90.6000018119812|1.2171898278934408|
|Technology|Africa| 322367.0430994034|  44129.4930000001|         2031|143.1999975964427|13.689207363046899|
|Technology|  East| 264872.0816922188| 47439.55759999996|         1927|76.30000080168247|17.910365372189244|
+----------+------+------------------+------------------+-------------+-----------------+------------------+
only showing top 5 

In [0]:
#Rename Columns for Clarity
df_transformed = df_transformed.withColumnRenamed("Sales", "TotalSales") \
                                .withColumnRenamed("Profit", "TotalProfit")
                                
df_transformed.show(5)

+----------+------+------------------+------------------+-------------+-----------------+------------------+
|  Category|Region|        TotalSales|       TotalProfit|TotalQuantity|    TotalDiscount|      ProfitMargin|
+----------+------+------------------+------------------+-------------+-----------------+------------------+
| Furniture|Canada|10595.279964447021|           2613.24|           78|              0.0|24.664190174953884|
|Technology|  EMEA|  300854.583026886|17494.443000000036|         2259|189.1000056862831| 5.814916569988444|
| Furniture|  East| 205540.3473367691|         2501.8162|         2151| 90.6000018119812|1.2171898278934408|
|Technology|Africa| 322367.0430994034|  44129.4930000001|         2031|143.1999975964427|13.689207363046899|
|Technology|  East| 264872.0816922188| 47439.55759999996|         1927|76.30000080168247|17.910365372189244|
+----------+------+------------------+------------------+-------------+-----------------+------------------+
only showing top 5 

In [0]:
from pyspark.sql.functions import col, year, to_date

# Add a Year Column 
df = df.withColumn("Year", year(to_date(col("Order Date"), "MM/dd/yyyy")))

# Show the result
df_transformed.show(5)


+----------+------+------------------+------------------+-------------+-----------------+------------------+
|  Category|Region|        TotalSales|       TotalProfit|TotalQuantity|    TotalDiscount|      ProfitMargin|
+----------+------+------------------+------------------+-------------+-----------------+------------------+
| Furniture|Canada|10595.279964447021|           2613.24|           78|              0.0|24.664190174953884|
|Technology|  EMEA|  300854.583026886|17494.443000000036|         2259|189.1000056862831| 5.814916569988444|
| Furniture|  East| 205540.3473367691|         2501.8162|         2151| 90.6000018119812|1.2171898278934408|
|Technology|Africa| 322367.0430994034|  44129.4930000001|         2031|143.1999975964427|13.689207363046899|
|Technology|  East| 264872.0816922188| 47439.55759999996|         1927|76.30000080168247|17.910365372189244|
+----------+------+------------------+------------------+-------------+-----------------+------------------+
only showing top 5 

In [0]:
from pyspark.sql.functions import col, year, to_date, sum
#Filter the Data for a Specific Year

# Step 1: Add the Year Column to the Original DataFrame
df = df.withColumn("Year", year(to_date(col("Order Date"), "MM/dd/yyyy")))

# Step 2: Perform the aggregation including 'Year'
df_transformed = df.groupBy("Category", "Region", "Year") \
    .agg(
        sum("Sales").alias("TotalSales"),
        sum("Profit").alias("TotalProfit"),
        sum("Quantity").alias("TotalQuantity"),
        sum("Discount").alias("TotalDiscount")
    )

# Step 3: Filter the Data for a Specific Year
df_transformed = df_transformed.filter(col("Year") == 2012)

# Show the result
df_transformed.show(5)


+----------+-------+----+------------------+------------------+-------------+------------------+
|  Category| Region|Year|        TotalSales|       TotalProfit|TotalQuantity|     TotalDiscount|
+----------+-------+----+------------------+------------------+-------------+------------------+
| Furniture|Oceania|2012|100519.00800000002|          8623.818|        607.0| 26.19999999999998|
|Technology|  North|2012|126353.56300000004|25098.862999999998|        809.0|             7.378|
|Technology| Africa|2012|64734.582000000024| 6320.742000000001|        404.0| 36.69999999999999|
| Furniture| Canada|2012|           1600.68|            290.19|         16.0|               0.0|
|Technology|Oceania|2012| 89761.76700000005|         14203.827|        688.0|23.499999999999986|
+----------+-------+----+------------------+------------------+-------------+------------------+
only showing top 5 rows



In [0]:
#Sort Data by Total Sales
df_transformed = df_transformed.orderBy(col("TotalSales").desc())
df_transformed.show()

+---------------+--------------+----+------------------+-------------------+-------------+------------------+
|       Category|        Region|Year|        TotalSales|        TotalProfit|TotalQuantity|     TotalDiscount|
+---------------+--------------+----+------------------+-------------------+-------------+------------------+
|     Technology|       Central|2012|237291.81162000002| 31373.854319999966|       1791.0| 61.14199999999993|
|      Furniture|       Central|2012|183778.99949999998| 5231.0848000000005|      1876.94| 85.14000000000004|
|Office Supplies|       Central|2012|180068.29200000002| 26988.422600000013|     6003.084| 198.4000000000001|
|     Technology|         North|2012|126353.56300000004| 25098.862999999998|        809.0|             7.378|
|Office Supplies|         South|2012|116962.67200000002| 10905.974199999997|     3831.186|151.70000000000016|
|      Furniture|         South|2012|       106962.4015|  8656.693399999998|       1108.0| 38.94999999999999|
|     Tech

In [0]:
# Save the transformed data in DBFS
df_transformed.write.format("delta").mode("overwrite").saveAsTable("transformed_data")



In [0]:
df_transformed.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- TotalSales: double (nullable = true)
 |-- TotalProfit: double (nullable = true)
 |-- TotalQuantity: double (nullable = true)
 |-- TotalDiscount: double (nullable = true)

